# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch
import numpy as np

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
print('the number of twits {}'.format(len(twits['data'])))


the number of twits 1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from string import punctuation

def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    #print('original message {}  \n'.format(message))
    
    # Lowercase the twit message
    text = message.lower()
    #print('lower case \n')
    #print(text + '\n')
    
    
    # Replace URLs with a space in the message
    #print('replace URL \n')
    pattern = '(?P<url>https?://[^\s]+)'
    text = re.sub(pattern, " ", text)
    #print(text + '\n')
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    #print('replace $\n')
    text = text.replace('$','')
    #print(text + '\n')
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    #print('Replace StockTwits usernames with a space \n')
    text = re.sub(r'@([A-Za-z0-9_]+)', '', text)
    #print(text + '\n')
    
    # Replace everything not a letter with a space
    #print('Replace everything not a letter with a space\n')
    text = ''.join([c for c in text if c not in punctuation])
    #print(text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    #print('Tokenize by splitting the string on whitespace into a list of words\n')
    tokens = text.split()
    #print(tokens)
   
    
    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    #print(' Lemmatize words\n')
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(t,pos='v') for t in tokens if len(t)>1]
    #print(tokens)
    #print('\n\n\n')
   
    
    return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
# TODO Implement
import datetime
starttime = datetime.datetime.now()

tokenized =[preprocess(msg) for msg in messages]

endtime = datetime.datetime.now()
run_time=(endtime - starttime)
print(run_time.seconds)
print(tokenized[:5])

183
[['fitb', 'great', 'buy', 'at', '2600ill', 'wait'], ['msft'], ['staanalystalert', 'for', 'tdg', 'jefferies', 'maintain', 'with', 'rat', 'of', 'hold', 'set', 'target', 'price', 'at', 'usd', '35000', 'our', 'own', 'verdict', 'be', 'buy'], ['amd', 'hear', 'there’s', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'on', 'stocktwits'], ['amd', 'reveal', 'yourself']]


### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [7]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 
bow = Counter()

for words in tokenized:
    bow.update(words)

bow

Counter({'fitb': 686,
         'great': 17288,
         'buy': 114091,
         'at': 137017,
         '2600ill': 1,
         'wait': 18580,
         'msft': 32270,
         'staanalystalert': 4072,
         'for': 273082,
         'tdg': 501,
         'jefferies': 928,
         'maintain': 1839,
         'with': 68661,
         'rat': 17629,
         'of': 211041,
         'hold': 39077,
         'set': 9535,
         'target': 16676,
         'price': 38981,
         'usd': 2861,
         '35000': 100,
         'our': 12126,
         'own': 10844,
         'verdict': 3735,
         'be': 579208,
         'amd': 261580,
         'hear': 4618,
         'there’s': 1074,
         'guy': 13082,
         'who': 15932,
         'know': 22421,
         'someone': 4982,
         'think': 31820,
         'somebody': 1047,
         'something': 5822,
         'on': 240995,
         'stocktwits': 1292,
         'reveal': 483,
         'yourself': 1183,
         'aapl': 105078,
         'why': 21

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [8]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
word_count = len([word for sentince in tokenized for word in sentince])

freqs = {key: val/word_count for key, val in bow.items()}


# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.000007

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 16

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = dict(bow.most_common(high_cutoff))

#from tqdm import tqdm as tqdm

filtered_words = [key for key in freqs if (key not in K_most_common and freqs[key] > low_cutoff)]

print(K_most_common)
print('{}'.format('\n'*3))
print(filtered_words[:30])
len(filtered_words)

{'be': 579208, 'the': 396909, 'to': 378535, 'for': 273082, 'amd': 261580, 'on': 240995, 'of': 211041, 'and': 207171, 'in': 202670, 'this': 201667, 'at': 137017, 'it': 133437, 'mu': 133133, 'amzn': 131902, 'will': 128057, 'fb': 123314}




['fitb', 'great', 'buy', 'wait', 'msft', 'staanalystalert', 'tdg', 'jefferies', 'maintain', 'with', 'rat', 'hold', 'set', 'target', 'price', 'usd', 'our', 'own', 'verdict', 'hear', 'there’s', 'guy', 'who', 'know', 'someone', 'think', 'somebody', 'something', 'stocktwits', 'reveal']


6517

## Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [9]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
sorted_vocab = set(sorted(filtered_words))

vocab = {word: ii for ii, word in enumerate(sorted_vocab)}

# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {value: key for key, value in vocab.items()}

# tokenized with the words not in `filtered_words` removed.
retained_word = set(list(id2vocab.values()))


starttime = datetime.datetime.now()

####
filtered = [[val for val in token if val in retained_word] for token in tokenized]
#####

endtime = datetime.datetime.now()
run_time=(endtime - starttime)
print('run_time={}'.format(run_time.seconds))

print(len(retained_word))
print('\n\n')
print(len(filtered))
tokenized == filtered

filtered[:1]

run_time=7
6517



1548010


[['fitb', 'great', 'buy', 'wait']]

In [10]:
tokenized == filtered

False

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [11]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [12]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.1968638695670079

Finally let's convert our tokens into integer ids which we can pass to the network.

In [13]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

In [14]:
len(token_ids)

1044153

In [15]:
len(sentiments)

1044153

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [16]:
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


In [17]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size ### hidden dim
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(self.vocab_size, self.embed_size)
        
        # Setup additional layers
        ## LTSM layer
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers,dropout=dropout, batch_first=False)
        ## dropout layer
        self.dropout = nn.Dropout(dropout)
        ## linear and logsoftmax layer
        self.fc = nn.Linear(lstm_size, output_size)
        self.logsoftmax = nn.LogSoftmax(dim=1)


    def forward(self, nn_input, hidden):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        # TODO Implement 
        batch_size = nn_input.size(0)
        
        x = nn_input.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs
        lstm_out = lstm_out[-1,:,:]
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        log_ps = self.logsoftmax(out)

        return log_ps, hidden

    
    
    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        Parameters
        ----------
            batch_size : The size of batches.
        Returns
        -------
            hidden_state
        """
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
        else:
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        return hidden

### View Model

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input_data = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)
model.cuda()
input_data = input_data.to(device)

logps, _ = model.forward(input_data, hidden)
print(logps)
print(torch.exp(logps))

tensor([[-1.5399, -1.9083, -1.8692, -1.4219, -1.4198],
        [-1.5578, -1.9216, -1.8878, -1.4099, -1.3965],
        [-1.4890, -1.9514, -1.9666, -1.4098, -1.3936],
        [-1.4829, -1.9657, -1.9438, -1.4323, -1.3821]], device='cuda:0')
tensor([[ 0.2144,  0.1483,  0.1543,  0.2413,  0.2418],
        [ 0.2106,  0.1464,  0.1514,  0.2442,  0.2475],
        [ 0.2256,  0.1421,  0.1399,  0.2442,  0.2482],
        [ 0.2270,  0.1401,  0.1432,  0.2388,  0.2510]], device='cuda:0')


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [19]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=True):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

a =dataloader(token_ids[:40],sentiments[:4])
next(a)

In [20]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 
split_frac = 0.8
split_idx = int(len(token_ids)*split_frac)

train_features = token_ids[:split_idx] 
valid_features = token_ids[split_idx:]
train_labels = sentiments[:split_idx]
valid_labels = sentiments[split_idx:]

In [21]:
len(valid_features)

208831

In [22]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
model.cuda()
text_batch = text_batch.to(device)

logps, hidden = model.forward(text_batch, hidden)

print(logps, hidden)

tensor([[-1.6228, -1.8490, -1.4234, -1.5513, -1.6483],
        [-1.5855, -1.8754, -1.5441, -1.5132, -1.5695],
        [-1.5110, -1.7759, -1.6039, -1.5059, -1.6765],
        [-1.3685, -1.6759, -1.7630, -1.5971, -1.6909],
        [-1.5840, -1.5774, -1.6183, -1.5633, -1.7111],
        [-1.4062, -1.7300, -1.5697, -1.5271, -1.8815],
        [-1.4664, -1.6612, -1.6305, -1.6717, -1.6317],
        [-1.5400, -1.6446, -1.7424, -1.4905, -1.6495],
        [-1.4468, -1.6794, -1.5813, -1.5544, -1.8253],
        [-1.5619, -1.6491, -1.5399, -1.5606, -1.7509],
        [-1.5354, -1.7641, -1.5959, -1.4929, -1.6830],
        [-1.4996, -1.6945, -1.5857, -1.6016, -1.6782],
        [-1.6012, -1.7715, -1.5446, -1.6215, -1.5266],
        [-1.5096, -1.7426, -1.6126, -1.4634, -1.7539],
        [-1.4565, -1.7153, -1.6499, -1.5304, -1.7230],
        [-1.5686, -1.6469, -1.6458, -1.4757, -1.7287],
        [-1.4540, -1.7498, -1.7058, -1.5747, -1.5904],
        [-1.3064, -1.8520, -1.6096, -1.5529, -1.8282],
        [-

### Training
It's time to train the neural network!

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 256,128 , 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(6518, 256)
  (lstm): LSTM(256, 128, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=128, out_features=5, bias=True)
  (logsoftmax): LogSoftmax()
)

In [24]:
device

device(type='cuda')

In [25]:
"""Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

starttime = datetime.datetime.now()


epochs = 1
batch_size = 10
learning_rate = 0.001

#Clipping parameter
clip = 5

print_every = 5
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# move model to GPU, if available
if(train_on_gpu):
    model.cuda()

model.train()

# train for some number of epochs
for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    

    
    steps = 0
    for text_batch, labels in dataloader(train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1
        
        # initialize hidden state
        hidden = model.init_hidden(labels.shape[0])
        #print(hidden)
      
       
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        model.zero_grad()
        output, hidden = model(text_batch, hidden)
        
        loss = criterion(output.squeeze(), labels)
        print('trian_loss={}'.format(loss))
        loss.backward()
        
        #Clip gradient
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        ##loss stats
        number_of_correct = 0
        if steps % print_every == 0:
            #get validation loss
            print('validation phase')
            val_h = model.init_hidden(labels.shape[0])
            val_losses = []
            model.eval()
            for inputs, labels in dataloader(valid_features,
                                             valid_labels,
                                             batch_size = labels.shape[0],
                                             sequence_length=20, 
                                             shuffle=True):
                #val_h = tuple([each.data for each in val_h])
                
                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()
                    
                output_val, val_h = model(inputs, val_h)
                val_loss = criterion(output_val.squeeze(), labels)
                #print('val_loss={}'.format(val_loss))
                val_losses.append(val_loss.item())                

            # TODO Implement: Print metrics
            model.train()
            
            print("Epoch: {}/{}...".format(epoch + 1, epochs),
                  "Step: {}...".format(steps),
                  "Training Loss: {:.6f}...".format(loss.item()),
                  "Validation Loss: {:.6f}".format(np.mean(val_losses)))

            
            
print("Training completed.")

endtime = datetime.datetime.now()
run_time=(endtime - starttime)
print(run_time.seconds)

Starting epoch 1
trian_loss=1.5780317783355713
trian_loss=1.5807409286499023
trian_loss=1.6134862899780273
trian_loss=1.5826107263565063
trian_loss=1.5871306657791138
validation phase


RuntimeError: Expected hidden[0] size (2, 1, 128), got (2, 10, 128)

In [ ]:
torch.cuda.is_available()

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [ ]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = pass
    # Convert words to ids
    tokens = pass
        
    # Adding a batch dimension
    text_input = pass
    # Get the NN output
    hidden = pass
    logps, _ = pass
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = pass
    
    return pred

In [ ]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [ ]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [ ]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

Using the `prediction` function, let's apply it to a stream of twits.

In [ ]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [ ]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.